In [ ]:
import tensorflow as tf

In [ ]:
import os
import zipfile
import keras_preprocessing
from keras_preprocessing import image
from tensorflow.keras import layers
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision
from tensorflow.keras.optimizers import RMSprop

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('val_accuracy')>=0.9700) and (logs.get('val_precision')>=0.9775) and (logs.get('val_mae')<=0.0385)):
      print("\nReached 97% accuracy, 97.75% precision, and less than .0385 MAE, so cancelling training!")
      self.model.stop_training = True

In [ ]:
callbacks = myCallback()
precision = Precision()

In [ ]:
#This block gets the pre-trained model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
x = layers.Dense(644, activation='swish')(x)
x = layers.Dropout(0.33)(x)
x = layers.Dense(644, activation='swish')(x)
x = layers.Dropout(0.33)(x) 
x = layers.Dense(124, activation='swish')(x) 
x = layers.Dropout(0.33)(x)    
x = layers.Dense(62, activation='swish')(x) 
x = layers.Dropout(0.33)(x)               
x = layers.Dense(1, activation='sigmoid')(x) 

model1 = Model(pre_trained_model.input, x) 

In [ ]:
#next, create a custom optimizer
optimizer1 = RMSprop(lr=0.0001)


In [ ]:
model1.compile(loss='binary_crossentropy',
              optimizer=optimizer1,
              metrics=['mae','accuracy', precision])

history1 = model1.fit(train_generator, 
                      validation_data = validation_generator,
                      epochs=40, 
                      steps_per_epoch = 100,
                      validation_steps = 50,
                      verbose = 2,
                       callbacks=[callbacks])